In [1]:
import sys 
import os 
sys.path.insert(0, os.path.dirname(os.getcwd()))

In [3]:
from utils import get_connection
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [4]:
con = get_connection()

connecting to rm-uf6lfxk83h7onjmwvjo.mysql.rds.aliyuncs.com3306/


In [5]:
pd.read_sql("show databases ",con)


,Database
0,douyin
1,information_schema
2,mysql
3,performance_schema
4,validation
5,xiaohongshu


In [17]:
tag_ls = pd.read_sql("SELECT distinct tag_new class_name FROM validation.tag_group_text_topic_validation" ,con)['class_name'].tolist()
tag_ls

['旅行',
 '其他',
 '广告与推广',
 '美食分享与推荐',
 '烹饪与食谱',
 '健康与饮食',
 '家庭日常',
 '户外活动',
 '节日庆祝',
 '怀旧与回忆',
 '社交互动',
 '健康与养生',
 '食物评测',
 '体重管理',
 '孕期饮食与健康',
 '学习与教育',
 '儿童饮食与健康',
 '日常记录',
 '运动健身',
 '校园生活',
 '生活感悟']

In [14]:
data = pd.read_csv('data/validation_3000.csv')
data.head()

,id,content,output,text_topic
0,e04592c0da33650d9c837464dca6c706|b9393c69e2e6d...,一拖二的周六，一早起床，勿勿忙忙准备好小娃需要看病的资料，再准备我需要摆摊的工具，大娃补课催...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '日常生活挑战...",日常生活挑战
1,18336dc645f26e3c3944e8c47c1c9831|d035d1c7e67d6...,津铺子的这个0卡果冻回购很多次了，一共四种口味：白桃、青提、荔枝和芒果。四个口味都还不错，口...,"{'食品类别': '蜜饯果干', '是否为零售食品': '是', '文本主题': '产品推荐...",产品推荐
2,b9240d65115b682962187bac31a5a14d|e64571aeb3fbf...,话梅柠檬茶！建议女孩子都把奶茶换成它！ 全网安利的咸柠茶！我能喝一整个夏天！真的好喝到原地转...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '饮品推荐',...",饮品推荐
3,a0e6ebe573ea14303727123a5abbef3e|de754b5d37397...,随手买的香辣鱼尾也太好吃了吧！鱼肉很有嚼劲，麻麻辣辣的也太过瘾了，平时追剧或者嘴馋了来一包，...,"{'食品类别': '肉脯卤味', '是否为零售食品': '是', '文本主题': '食品体验...",食品体验
4,d3fc3728a8c3e13ae5988aa15e4bb0fd|320b8f65d7782...,#踏春赏花最美的景点推荐 #感受大自然的气息和美景 #旅行推荐官 #抖音带你去赏花 #春暖花...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '旅游推荐',...",旅游推荐


## Embedding

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [ ]:
def cosine_similarity(X, Y):
    # 计算X与Y的点积
    dot_product = np.dot(X, Y.T)
    
    # 计算X和Y的范数
    X_norm = np.linalg.norm(X, axis=1, keepdims=True)
    Y_norm = np.linalg.norm(Y, axis=1)
    
    # 计算余弦相似度
    cosine_similarities = dot_product / (X_norm * Y_norm)
    
    return cosine_similarities

In [37]:
ref_ls = tag_ls
embed_model = model 
data = data 
content_col = 'content' 


tag_embed = embed_model.encode(ref_ls)

content_embed = embed_model.encode(data[content_col])



similarities = cosine_similarity(content_embed, tag_embed)



In [49]:



max_idx = np.argmax(similarities, axis = 1) 

max_similarity  = [(ref_ls[i], s[i]) for s,i in zip(similarities, max_idx)]


data[['ref', 'score']] = max_similarity

data


,id,content,output,text_topic,ref,score
0,e04592c0da33650d9c837464dca6c706|b9393c69e2e6d...,一拖二的周六，一早起床，勿勿忙忙准备好小娃需要看病的资料，再准备我需要摆摊的工具，大娃补课催...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '日常生活挑战...",日常生活挑战,家庭日常,0.462409
1,18336dc645f26e3c3944e8c47c1c9831|d035d1c7e67d6...,津铺子的这个0卡果冻回购很多次了，一共四种口味：白桃、青提、荔枝和芒果。四个口味都还不错，口...,"{'食品类别': '蜜饯果干', '是否为零售食品': '是', '文本主题': '产品推荐...",产品推荐,食物评测,0.487512
2,b9240d65115b682962187bac31a5a14d|e64571aeb3fbf...,话梅柠檬茶！建议女孩子都把奶茶换成它！ 全网安利的咸柠茶！我能喝一整个夏天！真的好喝到原地转...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '饮品推荐',...",饮品推荐,美食分享与推荐,0.342774
3,a0e6ebe573ea14303727123a5abbef3e|de754b5d37397...,随手买的香辣鱼尾也太好吃了吧！鱼肉很有嚼劲，麻麻辣辣的也太过瘾了，平时追剧或者嘴馋了来一包，...,"{'食品类别': '肉脯卤味', '是否为零售食品': '是', '文本主题': '食品体验...",食品体验,烹饪与食谱,0.510388
4,d3fc3728a8c3e13ae5988aa15e4bb0fd|320b8f65d7782...,#踏春赏花最美的景点推荐 #感受大自然的气息和美景 #旅行推荐官 #抖音带你去赏花 #春暖花...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '旅游推荐',...",旅游推荐,户外活动,0.493793
...,...,...,...,...,...,...
2995,594ac51b1657f4738586e8ae24552b52|2be16c8d4707b...,#好吃又便宜的网红零食推荐 发现一款超好吃的小零食，就是这款九块九三十包的酥牛排，还是盐津铺...,"{'食品类别': '肉脯卤味', '是否为零售食品': '是', '文本主题': '零食推荐...",零食推荐,美食分享与推荐,0.569377
2996,357b96bad7f1fa478a787ae85785d175|6298eda597a46...,救命这个燕麦酥真的不要太好吃！\n这款燕麦巧克力酥不含反式脂肪，外皮包裹巧克力\n再配上饱满...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '零食推荐',...",零食推荐,美食分享与推荐,0.511207
2997,956e86edcfd1f8201401df05c29181f3|846599ba7abdd...,尊敬的顾客朋友们\n🔔加一加水果零食明天4.18举行年会\n🙋🏻‍♀️全体员工参加⚠️闭店一...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '商业通知',...",商业通知,节日庆祝,0.459986
2998,7a6c58420d0aab1b5fc6f51a583503fe|9451fadf499ed...,#女大学生 #淄博烧烤 去淄博喽，八大局去了，紫米饼没吃，炒锅饼没吃，牛奶棒没吃，小蛋糕没吃...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '旅行体验',...",旅行体验,校园生活,0.444084
